In [3]:
from pyspark import SparkContext
sc = SparkContext()

22/05/07 19:39:36 WARN Utils: Your hostname, Equipo resolves to a loopback address: 127.0.1.1; using 192.168.1.20 instead (on interface wlo1)
22/05/07 19:39:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/07 19:39:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
import json
import datetime

In [5]:
def mapper(line):
    data = json.loads(line)
    usert = data['user_type']
    time = data['travel_time']
    date = convert_date(data['unplug_hourTime'])
    return usert, time, date

In [6]:
def convert_date(date):
    if isinstance(date, str):
        date = date.replace("Z","+0000")
        date = datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%S%z")
    else:
        date = date['$date']
        date = datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%S.%f%z")
    return (date.month,date.hour)

In [6]:
convert_date({ "$date" : "2019-06-01T12:40:00.000+0200" })

(6, 12)

In [7]:
rdd_base = sc.textFile('sample_10e3.json')

In [8]:
rdd = rdd_base.map(mapper)

In [9]:
rdd.countByKey()
#0: no ha sido posible determinarlo, 1: pase anual, 2: ocasional, 3: empresa

defaultdict(int, {1: 954, 2: 10, 3: 36})

In [10]:
rdd.groupByKey()
rdd.take(10)

[(1, 219, (6, 0)),
 (1, 359, (6, 0)),
 (1, 375, (6, 0)),
 (1, 264, (6, 0)),
 (1, 367, (6, 0)),
 (1, 174, (6, 0)),
 (1, 308, (6, 0)),
 (1, 462, (6, 0)),
 (1, 482, (6, 0)),
 (1, 480, (6, 0))]

In [11]:
rdd_user1 = rdd.filter(lambda x: x[0]==1).map(lambda x: ((x[2][0],x[2][1]),x[1]))#mes, hora 24h, tiempo seg.
rdd_user2 = rdd.filter(lambda x: x[0]==2).map(lambda x: ((x[2][0],x[2][1]),x[1]))
rdd_user3 = rdd.filter(lambda x: x[0]==3).map(lambda x: ((x[2][0],x[2][1]),x[1]))

In [12]:
rdd_user1.take(3)

[((6, 0), 219), ((6, 0), 359), ((6, 0), 375)]

In [13]:
rdd_user2.countByKey() #nº de viajes por horas del usuario tipo2

defaultdict(int, {(6, 0): 4, (6, 1): 6})

In [14]:
contador=(0,0)#tiempo medio del viaje de tipo2
rdd_user2.aggregateByKey(contador, lambda x,y: (x[0]+y, x[1]+1), lambda x,y: (x[0]+y[0], x[1]+y[1])).\
map(lambda x: (x[0], x[1][0]/x[1][1])).sortByKey().collect()

[((6, 0), 1046.25), ((6, 1), 710.3333333333334)]

In [26]:
s1 = sum(rdd_user1.map(lambda x: x[0]).take(3))
s2 = sum(rdd_user2.map(lambda x: x[0]).take(3))
s3 = sum(rdd_user3.map(lambda x: x[0]).take(3))
print(s1,s2,s3)

953 3093 3162
